In [1]:
from collections import defaultdict, OrderedDict
from operator import itemgetter
import numpy as np
import cv2
from colorthief import ColorThief
import io
from colorutils import Color
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
test_images = ['/Users/Rabbit/Downloads/china_test.jpg', 
               '/Users/Rabbit/Downloads/vilnius_test.jpg', 
               '/Users/Rabbit/Downloads/spices_test.jpg']

In [3]:
loc_fashion_images = ['/Users/Rabbit/capstone/materials/scrape_fashion_2/2003.86.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/2006.44.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/2006.56.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/2006.56.2.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/2007.30.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/2008.4.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/2010.98.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/74.32.4.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/76.64.1.png', 
                     '/Users/Rabbit/capstone/materials/scrape_fashion_2/80.1.4.png']

In [54]:
ColorThief?

In [4]:
# img_path = loc_fashion_images[1]

In [5]:
img_path = test_images[1]

In [20]:
class Palettization():
    """processes a single image path and returns image, centroids(dominant colors), palette"""
    
    def __init__(self, img_path):
        self.img_path = img_path
        self.rgb_palette = self.get_rgb_palette()
        self.hsv_palette = self.get_hsv_palette()
        self.color_arr = self.get_color_array()
        self.centroid_lst = self.arrayify_centroids()
        self.counts = self.centroid_cos_similar()

    def get_rgb_palette(self):
        """return image color palette"""
        colorthief = ColorThief(self.img_path)
        rgb_palette = colorthief.get_palette(color_count=9, quality=1)
        return rgb_palette


    def get_color_array(self):
        """return array of palette colors"""
        color_arr = []
        for color in self.rgb_palette:
            color_val = []
            for rgb_val in color:
                color_val.append(rgb_val / 255)
            color_arr.append(tuple(color_val))
        return color_arr


    def arrayify_centroids(self):
        """return array of image palette (centroids)"""
        self.centroid_lst = np.array(self.rgb_palette)
        return self.centroid_lst
    
    def _centroid_cos_similar(self, pixels, centroid):
        numerator = (pixels * centroid).sum(axis=1)
        denominator = (np.linalg.norm(pixels, axis=1) * np.linalg.norm(centroid))
        return (numerator / denominator)  


    def centroid_cos_similar(self):
        """"""
        lst = []
        self.centroid_lst = self.rgb_palette
        img = cv2.imread(self.img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        for centroid in self.centroid_lst:
            lst.append(self._centroid_cos_similar(img.reshape(-1, 3), centroid))

        centroid_argmax = np.array(lst).argmax(axis=0)

        _, counts = np.unique(centroid_argmax, return_counts=True)
        return counts

    
    def get_hsv_palette(self):
        palette_hsv = []
        for color in self.rgb_palette:
            c = Color(color)
            palette_hsv.append(c.hsv)
        return palette_hsv
    
    def zip_centroid_count(self):
        """"""
        return [{'rgb': rgb, 'count':count} for rgb, count in zip(self.rgb_palette, self.counts)]
    
    
    def display_palette(self):
        """"""
        self.get_color_array()
        return plt.imshow([(self.color_arr)])

    
    def display_img(self):
        """"""
        colorthief = ColorThief(self.img_path)
        img = colorthief.image
        return img

In [21]:
test_images

['/Users/Rabbit/Downloads/china_test.jpg',
 '/Users/Rabbit/Downloads/vilnius_test.jpg',
 '/Users/Rabbit/Downloads/spices_test.jpg']

In [49]:
def run_palettization_to_rgb_list(images):
    p_list = []
    for link in images:
        img = str(link)
        p = Palettization(img)
        p_list.append(p.get_rgb_palette())
    return p_list

In [ ]:
# def run_

In [50]:
run_palettization_to_rgb_list(test_images)

/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


[[(152, 173, 201),
  (33, 13, 12),
  (156, 14, 12),
  (170, 83, 69),
  (228, 228, 236),
  (145, 130, 127),
  (148, 98, 110),
  (124, 60, 132)],
 [(200, 161, 70),
  (36, 46, 40),
  (130, 146, 146),
  (171, 176, 172),
  (123, 145, 125),
  (109, 112, 101),
  (62, 85, 101),
  (73, 97, 67)],
 [(42, 28, 29),
  (228, 223, 204),
  (122, 56, 37),
  (172, 171, 171),
  (210, 165, 89),
  (210, 113, 29),
  (132, 122, 120),
  (157, 119, 81)]]

In [51]:
run_palettization_to_rgb_list(loc_fashion_images)

/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in true_divide


[[(234, 232, 230),
  (68, 60, 54),
  (149, 129, 98),
  (131, 98, 76),
  (150, 145, 132),
  (181, 157, 130),
  (198, 181, 152),
  (108, 132, 93)],
 [(208, 205, 207),
  (175, 136, 71),
  (67, 51, 44),
  (114, 81, 50),
  (168, 168, 171),
  (199, 168, 120),
  (122, 118, 117),
  (162, 136, 109)],
 [(121, 139, 157),
  (76, 36, 23),
  (133, 79, 57),
  (227, 220, 214),
  (174, 131, 113),
  (51, 68, 81),
  (185, 196, 212),
  (188, 204, 210)],
 [(143, 166, 184),
  (122, 102, 70),
  (88, 64, 41),
  (227, 219, 204),
  (152, 135, 108),
  (47, 35, 22),
  (188, 212, 231),
  (91, 119, 139)],
 [(106, 106, 105),
  (160, 123, 116),
  (100, 32, 34),
  (218, 220, 220),
  (192, 113, 111),
  (95, 63, 38),
  (41, 38, 35),
  (161, 172, 180)],
 [(118, 109, 110),
  (212, 155, 158),
  (119, 33, 41),
  (27, 8, 8),
  (173, 94, 100),
  (186, 189, 189),
  (181, 179, 188),
  (118, 52, 43)],
 [(177, 171, 172),
  (47, 51, 34),
  (88, 94, 70),
  (111, 111, 98),
  (226, 225, 226),
  (87, 72, 59),
  (133, 126, 115),
  (132

In [ ]:
# %pdb

In [13]:
c = Color((0,0,0))

In [17]:
c.hsv

(0, 0, 0.0)

In [ ]:
palette_me_now = Palettization(img_path)

In [ ]:
# %pdb

In [ ]:
palette_me_now.zip_centroid_count()

In [ ]:
# img_palette

In [ ]:
palette_me_now.get_hsv_palette()

In [ ]:
try_me = [Palettization(link) for link in test_images]

In [ ]:
i = 2
try_me[i].display_palette()

try_me[i].zip_centroid_count()

In [ ]:
try_me[i].display_img()

In [ ]:
for item in try_me:
    item.display_palette()

In [ ]:
palette_color_percent

In [ ]:
palette_color_percent = {}
# label_
img_pixel_sum = 0

for image in try_me:
    for i in image.zip_centroid_count():
        img_pixel_sum += i['count']
#     print(img_pixel_sum)
        palette_color_percent[i['rgb']] = i['count'] / 540000

sorted(palette_color_percent, key=lambda x: palette_color_percent[x], reverse=True)

In [ ]:
type(var)

In [ ]:
class SetSimilarity():
    """find set(palette) most 'central' of sets, consisting of items(rgb vector) """
    
    #def __init__(self, set_of_palettes):
    #    self.p_list= list_of_palettes
        
    def c2c_dist(self, color_a, color_b):
        """return the distance(float) between two items(rgb vectors)"""
        dist_c2c = np.linalg.norm(np.array(color_a) - np.array(color_b))
        return(dist_c2c)
                
    def c2p_dist(self, color_a, palette_b):
        """return minimum distance(float) between an item(rgb vector) and a set(palette)"""
        """find min dist between color_a and all colors in palet_b
        use c2c_dist"""
        dist_c2p_list = []
        for color in palette_b:
            dist = self.c2c_dist(color, color_a)
            dist_c2p_list.append(dist)
        dist_c2p = np.amin(dist_c2p_list)
        return(dist_c2p)
    
    def p2p_dist(self, palette_a, palette_b):
        """return sum(float) of c2p_dist between two sets(palette)"""
        # sum of c2p-dists
        dist_p2p_list = []
        for color in palette_a:
            dist_p2p_list.append(self.c2p_dist(color, palette_b))
            dist_p2p = np.sum(dist_p2p_list)
        return(dist_p2p)
    
    def set_central_list(self, list_of_palettes):
        """return most 'central' set(palette(s), if there are multiples)"""
        p_cent_list = []

        for palette_a in list_of_palettes:
            palette_a_centrality_list = []

            for palette in list_of_palettes:
                palette_a_centrality_list.append(self.p2p_dist(palette_a, palette))
                
            sum_temp = np.sum(palette_a_centrality_list)
            p_cent_list.append((sum_temp, palette_a))

        p_min = min(p_cent_list, key=itemgetter(0))[0]
        p_cent_list_srtd = sorted(p_cent_list, key=lambda x: x[0])
        
        p_central = [ x for x in p_cent_list_srtd if x[0]==p_min ]
        return(p_central)

In [ ]:
# p_1 = [ (200, 161, 70),
#         (36, 46, 40),
#         (156, 14, 12) ]

# p_2 = [ (152, 173, 201),
#         (172, 171, 171),
#         (122, 56, 37) ]

# p_3 = [ (123, 145, 125),
#         (228, 228, 236),
#         (210, 113, 29) ]

# p_3_a = [ (228, 228, 236),
#           (123, 145, 125),
#           (210, 113, 29) ]

# p_4 = [ (228, 223, 204),
#         (157, 119, 81),
#         (130, 146, 146),
#         (210, 165, 89) ]

# p_5 = [ (33, 13, 12),
#         (42, 28, 29),
#         (132, 122, 120),
#         (170, 83, 69),
#         (145, 130, 127),
#         (109, 112, 101) ]

# p_6 = [ (62, 85, 101),
#         (148, 98, 110),
#         (171, 176, 172),
#         (124, 60, 132),
#         (73, 97, 67) ]

# p_test_lst = [ p_1, p_2, p_3, p_3_a, p_4, p_5, p_6 ]

In [ ]:
# p_test = SetSimilarity()

In [ ]:
print(p_test.c2c_dist(p_1[0], p_1[2]))

print(p_test.c2p_dist(p_1[0], p_2))

print(p_test.p2p_dist(p_1, p_6))

print(p_test.set_central_list(p_test_lst))

In [ ]:
palette_1 = [ ( 255, 0, 0), (0, 255, 0) ]
palette_2 = [ (0, 255, 0), (0, 0, 255), (42, 142, 242), (108, 212, 228) ]
palette_3 = [ (255, 0, 0), (0, 0, 255) ]
palette_4 = [ (127,127,127), (0, 0, 255) ]
palette_5 = [ ( 255, 0, 0), (0, 255, 0) ]
palette_list_ex = [palette_1, palette_2, palette_3, palette_4, palette_5]
color_a = palette_1[0]
color_b = palette_1[1]

In [ ]:
test_other = SetSimilarity()

In [ ]:
print(test_other.c2c_dist(palette_2[0], palette_1[1]))

print(test_other.c2p_dist(color_a, palette_2))

print(test_other.p2p_dist(palette_2, palette_4))

print(test_other.set_central_list(palette_list_ex))

In [ ]:
from colour import Color

In [ ]:
c = Color()

In [ ]:
morning_pic = ['/Users/Rabbit/Desktop/Screen Shot 2017-10-25 at 07.56.08.png',
              '/Users/Rabbit/Desktop/Screen Shot 2017-10-25 at 08.08.44.png', 
              '/Users/Rabbit/Desktop/Screen Shot 2017-10-25 at 08.17.04.png',
              '/Users/Rabbit/Desktop/Screen Shot 2017-10-25 at 08.33.38.png']

In [ ]:
c_harmony = Palettization(morning_pic[0])

In [ ]:
c_harmony.display_palette()

In [ ]:
c_harmony.zip_centroid_count()

In [ ]:
c_harmony.display_img()

In [ ]:
import colorsys

In [ ]:
try_me[1].arrayify_centroids()

In [ ]:
get_palette(img_path)

In [ ]:
plt.imshow([(centroid_lst).astype('uint8')])

In [ ]:
import colorgram

img = (morning_pic[0])

In [ ]:
colors = colorgram.extract(img, 6)

In [ ]:
colors[0]

In [ ]:
# for color in palette_color_percent:
#     r_chan = color[0]
#     b_chan = color[1]
#     g_chan = color[2]
#     r_vals.append(r_chan)
#     b_vals.append(b_chan)
#     g_vals.append(g_chan)

def colorT_rgb2hsv(rgb_vector):
    """returns an hsv vector from rgb"""
    hsv_vector = []
    for color in rgb_vector:
        r_prime = color[0]/255
        b_prime = color[1]/255
        g_prime = color[2]/255
        
        

In [ ]:
import colorsys

def rgb_hsv(rgb_tup):
    """returns hsv tuple from rgb"""
    """thanks for your help Zach E.!"""
    

    r = rgb_tup[0]
    b = rgb_tup[1]
    g = rgb_tup[2]
#         print(r, g, b)
    h, s, v = colorsys.rgb_to_hsv(r / 255., g / 255., b / 255.)
    h = h * 360
    hsv = [h, s, v]
    return tuple(hsv)

In [ ]:
var

In [ ]:
rgb_hsv(var)

In [ ]:
_1 = np.uint8([[_var]])
hsv_1 = cv2.cvtColor(_1, cv2.COLOR_RGB2HSV)

In [ ]:
hsv_1

In [ ]:
green = np.uint8([[[0,255,0 ]]])
hsv_green = cv2.cvtColor(green,cv2.COLOR_RGB2HSV)
print(hsv_green)

In [ ]:
var = (108, 212, 228)

In [ ]:
from colorutils import Color

def get_hsv(rgb):
    c = Color(rgb)
    return c.hsv

In [ ]:
get_hsv(var)
    
    
    

In [ ]:
rgb2hsv(var)